## Initi


### Packages

In [1]:
import logging
import sys
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


### Define Logger

In [2]:

# Configure logger
def setup_logger(debug_mode=False):
    level = logging.DEBUG if debug_mode else logging.INFO
    
    logging.basicConfig(
        level=level,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler(sys.stdout)  # Print to console
        ]
    )
    return logging.getLogger(__name__)

    return logging.getLogger(__name__)

# Initialize logger
DEBUG = True  # Set to False in production
logger = setup_logger(debug_mode=DEBUG)
time_sleep=1


### define env variables

In [3]:

email = os.getenv('YOUR_SECRET_EMAIL')
password = os.getenv('YOUR_SECRET_PASSWORD')
logon_url = os.getenv("YOUR_SECRET_LOGON_URL")
planning_url = os.getenv('YOUR_SECRET_PLANNING_URL')
login_url = os.getenv('YOUR_SECRET_LOGIN_URL')
my_name = os.getenv('YOUR_SECRET_MY_NAME')


In [4]:

# Valeurs par défaut
DEFAULT_DATE = "19-Aug-25"
DEFAULT_TIME = "4:30"
DEFAULT_LEVEL = "Intermediate"

# Détection Jupyter
def is_jupyter():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True
        return False
    except NameError:
        return False

if is_jupyter():
    # Dans Jupyter: utiliser les valeurs par défaut
    TARGET_DATE = DEFAULT_DATE
    TARGET_TIME = DEFAULT_TIME
    COURSE_LEVEL = DEFAULT_LEVEL
else:
    # Dans un script normal: utiliser sys.argv
    TARGET_DATE = sys.argv[1] if len(sys.argv) > 1 else DEFAULT_DATE
    TARGET_TIME = sys.argv[2] if len(sys.argv) > 2 else DEFAULT_TIME
    COURSE_LEVEL = sys.argv[3] if len(sys.argv) > 3 else DEFAULT_LEVEL

print(f"📅 Date: {TARGET_DATE}")
print(f"🕐 Time: {TARGET_TIME}")
print(f"🎾 Level: {COURSE_LEVEL}")

📅 Date: 19-Aug-25
🕐 Time: 4:30
🎾 Level: Intermediate


In [5]:


# Vérification des secrets
logger.info("=== STATUS DES AUTRES SECRETS (MASQUÉS) ===")
logger.info(f"EMAIL: {'✅ DÉFINI' if email else '❌ MISSING'}")
logger.info(f"PASSWORD: {'✅ DÉFINI' if password else '❌ MISSING'}")
logger.info(f"MY_NAME: {'✅ DÉFINI' if my_name else '❌ MISSING'}")
logger.info(f"LOGON_URL: {'✅ DÉFINI' if logon_url else '❌ MISSING'}")
logger.info(f"PLANNING_URL: {'✅ DÉFINI' if planning_url else '❌ MISSING'}")
logger.info(f"LOGIN_URL: {'✅ DÉFINI' if login_url else '❌ MISSING'}")

# Vérification complète
secrets_dict = {
    "EMAIL": email,
    "PASSWORD": password,
    "MY_NAME": my_name,
    "LOGON_URL": logon_url,
    "PLANNING_URL": planning_url,
    "LOGIN_URL": login_url
}

missing_secrets = [name for name, value in secrets_dict.items() if not value]

if missing_secrets:
    logger.error("❌ ERREUR: UN OU PLUSIEURS SECRETS SONT MANQUANTS")
    logger.error("=== SECRETS MANQUANTS ===")
    for secret_name in missing_secrets:
        logger.error(f"{secret_name}: ❌ MISSING")
    logger.critical("ARRÊT DU PROGRAMME - VÉRIFIEZ VOS VARIABLES D'ENVIRONNEMENT")
    sys.exit(1)
else:
    logger.info("✅ TOUS LES SECRETS SONT DÉFINIS")
    logger.debug("Poursuite de l'exécution...")

2025-10-30 21:44:47,345 - __main__ - INFO - === STATUS DES AUTRES SECRETS (MASQUÉS) ===
2025-10-30 21:44:47,347 - __main__ - INFO - EMAIL: ✅ DÉFINI
2025-10-30 21:44:47,347 - __main__ - INFO - PASSWORD: ✅ DÉFINI
2025-10-30 21:44:47,348 - __main__ - INFO - MY_NAME: ✅ DÉFINI
2025-10-30 21:44:47,350 - __main__ - INFO - LOGON_URL: ✅ DÉFINI
2025-10-30 21:44:47,350 - __main__ - INFO - PLANNING_URL: ✅ DÉFINI
2025-10-30 21:44:47,351 - __main__ - INFO - LOGIN_URL: ✅ DÉFINI
2025-10-30 21:44:47,352 - __main__ - INFO - ✅ TOUS LES SECRETS SONT DÉFINIS
2025-10-30 21:44:47,353 - __main__ - DEBUG - Poursuite de l'exécution...


In [6]:

from datetime import datetime, timedelta


def check_date(target_date):
    # Parse the date string
    your_date = datetime.strptime(target_date, "%d-%b-%y")
    
    # Get today's date
    today = datetime.now()
    
    # Check if your date is after today
    is_after_today = your_date > today
    
    # Calculate the difference in days
    difference = your_date - today
    days_difference = difference.days
    
    # Check if exactly 7 days difference
    next_week = days_difference == 6
    less_7_days = days_difference <= 6
    possible = is_after_today & less_7_days
    
    logger.info(f"Your date: {your_date.strftime('%Y-%m-%d')}")
    logger.info(f"Today: {today.strftime('%Y-%m-%d')}")
    logger.info(f"Is after today: {is_after_today}")
    logger.info(f"Days difference: {days_difference}")
    
    if possible:
        logger.debug("✅ Date is possible (within next 7 days)")
    else:
        logger.warning("⚠️ Date is not possible (not within next 7 days)")
    
    if next_week:
        logger.debug("📅 Date is exactly next week")
    
    return possible, next_week

# Usage:
# possible, next_week = check_date("25-Aug-25")

## classe

In [7]:


class SecureWebLogin:
    """Classe pour gérer le login sécurisé avec driver comme objet"""
    
    def __init__(self, website_url, email_login, secret_password):
        self.website_url = website_url
        self.email_login = email_login
        self.secret_password = secret_password
        self.driver = None
        self._setup_driver()
    
    def _setup_driver(self):
        """Initialise le driver Chrome"""
        logger.debug("Initialisation du driver Chrome...")
        # Configuration
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        
        # Driver
        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=options)
        
        # Si vous avez déjà un driver ouvert
        #self.driver.set_window_size(1920, 1080)
        #self.driver.maximize_window()
        # Ou définir une taille spécifique
        #self.driver.set_window_size(1400, 900)
        logger.debug("Driver configuré avec succès")
    
    def login(self):
        """Script simple : saisir email + password et prendre screenshots"""
        logger.info("🚀 Script simple - Email + Password + Screenshots")
        logger.info("=" * 50)
        logger.debug(f"🌐 Site: {self.website_url}")
        logger.debug(f"📧 Email: {self.email_login}")
        logger.info("🔑 Password: ****")
        
        # Rafraîchir la page pour que les changements prennent effet
        self.driver.refresh()
        
        # Étape 1: Aller sur le site
        logger.info("🌐 Connexion au site...")
        self.driver.get(self.website_url)
        time.sleep(3)
        logger.info(f"📄 Titre: {self.driver.title}")
        
        # Screenshot avant
        #self.driver.save_screenshot("1_avant_saisie.png")
        logger.debug("📸 Screenshot 1: 1_avant_saisie.png")
        
        email_field = self.driver.find_element(By.XPATH, "//input[@type='email']")
        logger.info("✅ Champ email trouvé par type='email'")
        
        if email_field:
            logger.info("📝 Saisie de l'email...")
            email_field.clear()
            email_field.send_keys(self.email_login)
            logger.debug(f"✅ Email saisi: {self.email_login}")
            time.sleep(1)
            
            # Screenshot après email
            #self.driver.save_screenshot("2_apres_email.png")
            logger.debug("📸 Screenshot 2: 2_apres_email.png")
        
        # Étape 3: Trouver et remplir le champ password
        logger.info("🔑 Recherche du champ Password...")
        password_field = None
        password_field = self.driver.find_element(By.XPATH, "//input[@type='password']")
        logger.info("✅ Champ password trouvé par type='password'")
        
        if password_field:
            logger.info("📝 Saisie du password...")
            password_field.clear()
            password_field.send_keys(self.secret_password)
            logger.info("✅ Password saisi")
            time.sleep(1)
            
            # Screenshot après password
            #self.driver.save_screenshot("3_apres_password.png")
            logger.debug("📸 Screenshot 3: 3_apres_password.png")
        
        login_button = None
        login_button = self.driver.find_element(By.XPATH, "//input[@value='Login']")
        login_button.click()
        logger.info("✅ Bouton cliqué!")
        
        # Étape 6: Attendre la redirection et vérifier
        logger.info("⏳ Attente de la redirection...")
        time.sleep(5)
        
        # Screenshot après connexion
        #self.driver.save_screenshot("4_apres_clic.png")
        logger.debug("📸 Screenshot 4: 4_apres_clic.png")
        logger.debug(f"URL actuelle: {self.driver.current_url}")
        
        return self.driver
    
    def check_basket(self):
    # Look up if basket is empty
        if self.driver.current_url == login_url:
           is_empty = self.driver.execute_script("return document.querySelector('span.basket-badge').textContent;") == "0"
           return is_empty
        else:
            logger.info("not login")

    def go_planning(self):
        return self.driver.get(planning_url) #go to planning after login
    
    def get_driver(self):
        """Retourne le driver pour utilisation externe"""
        return self.driver

    def quit(self):
        """Ferme le driver"""
        if self.driver:
            self.driver.quit()
            logger.info("🔒 Driver fermé")
            self.is_logged_in = False
    
    

# Usage:
# secure_login = SecureWebLogin("https://site.com", "email@test.com", "password123")
# secure_login.login()
# 
# Utiliser le driver dans d'autres fonctions:
# driver = secure_login.driver

driver = access_login(logon_url, email, password) # navigate after login


In [19]:
possible_to_book

False

In [ ]:
possible_to_book, next_week_booking = check_date(TARGET_DATE)
    

2025-10-30 21:35:07,182 - __main__ - INFO - Your date: 2025-08-19
2025-10-30 21:35:07,185 - __main__ - INFO - Today: 2025-10-30
2025-10-30 21:35:07,186 - __main__ - INFO - Is after today: False
2025-10-30 21:35:07,187 - __main__ - INFO - Days difference: -73
2025-10-30 21:35:07,188 - __main__ - WARNING - ⚠️ Date is not possible (not within next 7 days)


TypeError: 'module' object is not callable

si pas possible to book print info mauvaise date et sort proprement sinon on continue

In [9]:
secure_login = SecureWebLogin(logon_url, email, password)
#secure_login. .save_screenshot("2_apres_email.png")
secure_login.get_driver().save_screenshot("test.png")


2025-10-30 21:45:40,385 - __main__ - DEBUG - Initialisation du driver Chrome...
2025-10-30 21:45:40,387 - WDM - INFO - ====== WebDriver manager ======
2025-10-30 21:45:40,478 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-10-30 21:45:40,480 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): googlechromelabs.github.io:443
2025-10-30 21:45:40,510 - urllib3.connectionpool - DEBUG - https://googlechromelabs.github.io:443 "GET /chrome-for-testing/latest-patch-versions-per-build.json HTTP/1.1" 200 13861
2025-10-30 21:45:40,515 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-10-30 21:45:40,517 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): googlechromelabs.github.io:443
2025-10-30 21:45:40,545 - urllib3.connectionpool - DEBUG - https://googlechromelabs.github.io:443 "GET /chrome-for-testing/latest-patch-versions-per-build.json HTTP/1.1" 200 13861
2025-10-30 21:45:40,549 - WDM - INFO - Driver [/home/vscode/.

True

In [11]:
secure_login.get_driver().save_screenshot("test.png")


2025-10-30 21:46:28,689 - selenium.webdriver.remote.remote_connection - DEBUG - GET http://localhost:60059/session/e98918433d9d8cbfc435aa9c7ab2cb06/screenshot {}
2025-10-30 21:46:28,724 - urllib3.connectionpool - DEBUG - http://localhost:60059 "GET /session/e98918433d9d8cbfc435aa9c7ab2cb06/screenshot HTTP/1.1" 200 0
2025-10-30 21:46:28,725 - selenium.webdriver.remote.remote_connection - DEBUG - Remote response: status=200 | data={"value":"..."} | headers=HTTPHeaderDict({'Content-Length': '45708', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2025-10-30 21:46:28,727 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request


True

In [ ]:
secure_login.save_

In [17]:
secure_login.quit()

2025-10-30 21:33:58,627 - selenium.webdriver.remote.remote_connection - DEBUG - DELETE http://localhost:39115/session/ac5b5ae453647b015c589cf02647494c {}
2025-10-30 21:33:58,686 - urllib3.connectionpool - DEBUG - http://localhost:39115 "DELETE /session/ac5b5ae453647b015c589cf02647494c HTTP/1.1" 200 0
2025-10-30 21:33:58,687 - selenium.webdriver.remote.remote_connection - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2025-10-30 21:33:58,687 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request
2025-10-30 21:33:58,703 - __main__ - INFO - 🔒 Driver fermé


2025-10-30 21:26:07,749 - __main__ - INFO - Your date: 2025-08-19
2025-10-30 21:26:07,752 - __main__ - INFO - Today: 2025-10-30
2025-10-30 21:26:07,753 - __main__ - INFO - Is after today: False
2025-10-30 21:26:07,754 - __main__ - INFO - Days difference: -73
2025-10-30 21:26:07,755 - __main__ - WARNING - ⚠️ Date is not possible (not within next 7 days)


## WIP

In [10]:



#load_dotenv()  # Charge le fichier .env when running in codespace

import time
#from PIL import Image
import io

def take_full_page_screenshot(driver, filename):
    # Obtenir la taille de la fenêtre
    window_size = driver.get_window_size()
    
    # Obtenir la hauteur totale de la page
    total_height = driver.execute_script("return document.body.parentNode.scrollHeight")
    
    # Redimensionner pour capturer toute la page
    driver.set_window_size(window_size['width'], total_height)
    
    # Attendre que la page se charge complètement
    time.sleep(2)
    
    # Prendre la capture d'écran
    driver.save_screenshot(filename)
    
    # Remettre la taille originale
    driver.set_window_size(window_size['width'], window_size['height'])


In [ ]:


def access_login(website_url, email_login, secret_password):
    """Script simple : saisir email + password et prendre screenshots"""

    print("🚀 Script simple - Email + Password + Screenshots")
    print("=" * 50)
    #print(f"🌐 Site: {website_url}")
    #print(f"📧 Email: {email}")
    print("🔑 Password: ****")
    print()

    # Configuration
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')

    # Driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

        # Si vous avez déjà un driver ouvert
    driver.set_window_size(1920, 1080)
    driver.maximize_window()

    # Ou définir une taille spécifique
    driver.set_window_size(1400, 900)

    # Rafraîchir la page pour que les changements prennent effet
    driver.refresh()

    # Étape 1: Aller sur le site
    print("🌐 Connexion au site...")
    driver.get(website_url)
    time.sleep(3)
    print(f"📄 Titre: {driver.title}")
    # Screenshot avant
    #driver.save_screenshot("1_avant_saisie.png")
    print("📸 Screenshot 1: 1_avant_saisie.png")
    email_field = driver.find_element(By.XPATH, "//input[@type='email']")
    ("✅ Champ email trouvé par type='email'")
    email_field
    if email_field:
            print("📝 Saisie de l'email...")
            email_field.clear()
            email_field.send_keys(email_login)
            #(f"✅ Email saisi: {email_login}")
            time.sleep(1)

    # Screenshot après email
    #driver.save_screenshot("2_apres_email.png")
    print("📸 Screenshot 2: 2_apres_email.png")
    # Étape 3: Trouver et remplir le champ password
    print("🔑 Recherche du champ Password...")

    password_field = None

    password_field = driver.find_element(By.XPATH, "//input[@type='password']")
    print("✅ Champ password trouvé par type='password'")


    if password_field:
        print("📝 Saisie du password...")
        password_field.clear()
        password_field.send_keys(secret_password)
        print("✅ Password saisi")
        time.sleep(1)

        # Screenshot après password
    #driver.save_screenshot("3_apres_password.png")
    print("📸 Screenshot 3: 3_apres_password.png")

    login_button = None
    login_button = driver.find_element(By.XPATH, "//input[@value='Login']")

    login_button.click()
    print("✅ Bouton cliqué!")

    # Étape 6: Attendre la redirection et vérifier
    print("⏳ Attente de la redirection...")
    time.sleep(5)

    # Screenshot après connexion
    #driver.save_screenshot("4_apres_clic.png")
    print("📸 Screenshot 4: 4_apres_clic.png")
    #print(driver.current_url)
    return driver


In [11]:
import logging
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

class SecureWebLogin:
    """Handles secure web login with Selenium WebDriver"""
    
    def __init__(self, headless=True, window_size=(1400, 900)):
        """Initialize the web driver with specified options"""
        self.logger = logging.getLogger(self.__class__.__name__)
        self.driver = None
        self.headless = headless
        self.window_size = window_size
        self._initialize_driver()
    
    def _initialize_driver(self):
        """Initialize Chrome driver with options"""
        self.logger.info("🚀 Initializing Chrome driver...")
        
        options = Options()
        if self.headless:
            options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        
        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=options)
        self.driver.set_window_size(*self.window_size)
        
        self.logger.info(f"✅ Driver initialized (window size: {self.window_size})")
    
    def login(self, website_url, email_login, secret_password):
        """Perform login on the website"""
        self.logger.info("=" * 50)
        self.logger.info("🔑 Starting login process...")
        
        try:
            # Navigate to website
            self.logger.info(f"🌐 Navigating to: {website_url}")
            self.driver.get(website_url)
            time.sleep(3)
            self.logger.info(f"📄 Page title: {self.driver.title}")
            
            # Find and fill email field
            self.logger.info("📝 Entering email...")
            email_field = self.driver.find_element(By.XPATH, "//input[@type='email']")
            email_field.clear()
            email_field.send_keys(email_login)
            self.logger.info("✅ Email entered")
            time.sleep(1)
            
            # Find and fill password field
            self.logger.info("🔑 Entering password...")
            password_field = self.driver.find_element(By.XPATH, "//input[@type='password']")
            password_field.clear()
            password_field.send_keys(secret_password)
            self.logger.info("✅ Password entered")
            time.sleep(1)
            
            # Click login button
            self.logger.info("🔘 Clicking login button...")
            login_button = self.driver.find_element(By.XPATH, "//input[@value='Login']")
            login_button.click()
            self.logger.info("✅ Login button clicked")
            
            # Wait for redirect
            self.logger.info("⏳ Waiting for redirection...")
            time.sleep(5)
            
            self.logger.info(f"✅ Login complete. Current URL: {self.driver.current_url}")
            return True
            
        except Exception as e:
            self.logger.error(f"❌ Login failed: {str(e)}")
            return False
    
    def take_screenshot(self, filename):
        """Take a screenshot"""
        self.driver.save_screenshot(filename)
        self.logger.info(f"📸 Screenshot saved: {filename}")
    
    def get_driver(self):
        """Return the driver for use in other functions"""
        return self.driver
    
    def quit(self):
        """Close the driver"""
        if self.driver:
            self.driver.quit()
            self.logger.info("🔒 Driver closed")

web_login = SecureWebLogin(headless=False)
    
# Perform login
success = web_login.login(
        website_url=logon_url,
        email_login=email,
        secret_password=password
    )
    
if success:
        # Get driver for other operations
        driver = web_login.get_driver()
        
        # Use driver for other tasks...
        # other_function(driver)
        
    # Clean up
web_login.quit()



2025-10-30 20:59:03,957 - SecureWebLogin - INFO - 🚀 Initializing Chrome driver...
2025-10-30 20:59:03,957 - WDM - INFO - ====== WebDriver manager ======
2025-10-30 20:59:04,070 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-10-30 20:59:04,072 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): googlechromelabs.github.io:443
2025-10-30 20:59:04,111 - urllib3.connectionpool - DEBUG - https://googlechromelabs.github.io:443 "GET /chrome-for-testing/latest-patch-versions-per-build.json HTTP/1.1" 200 13863
2025-10-30 20:59:04,116 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-10-30 20:59:04,117 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): googlechromelabs.github.io:443
2025-10-30 20:59:04,146 - urllib3.connectionpool - DEBUG - https://googlechromelabs.github.io:443 "GET /chrome-for-testing/latest-patch-versions-per-build.json HTTP/1.1" 200 13863
2025-10-30 20:59:04,150 - WDM - INFO - Driver [/home/vscode

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x62d77a7b5fea <unknown>
#1 0x62d77a233d06 <unknown>
#2 0x62d77a26f32c <unknown>
#3 0x62d77a26af94 <unknown>
#4 0x62d77a2ba3b3 <unknown>
#5 0x62d77a2b9a66 <unknown>
#6 0x62d77a2abbe3 <unknown>
#7 0x62d77a2782d2 <unknown>
#8 0x62d77a278f91 <unknown>
#9 0x62d77a7799e8 <unknown>
#10 0x62d77a77d84f <unknown>
#11 0x62d77a760ec9 <unknown>
#12 0x62d77a77e3f5 <unknown>
#13 0x62d77a74674f <unknown>
#14 0x62d77a7a2cc8 <unknown>
#15 0x62d77a7a2ea3 <unknown>
#16 0x62d77a7b4f83 <unknown>
#17 0x75381f592b7b <unknown>


In [ ]:

def check_basket(driver,login_url):
    # Look up if basket is empty
    if driver.current_url == login_url:
        is_empty = driver.execute_script("return document.querySelector('span.basket-badge').textContent;") == "0"
        return is_empty
    else:
         print("not login")
    
def click_for_me(driver, target_date, my_name):
    moi_elements = driver.find_elements(By.XPATH, f"//*[contains(text(), '{my_name}')]")
    # Chercher un bouton près de moi
    for i, moi_element in enumerate(moi_elements):

        # Chercher dans le conteneur parent
        parent = moi_element.find_element(By.XPATH, "./..")
        
        # Chercher des boutons de réservation
        moi_buttons = parent.find_elements(By.XPATH, ".//*[contains(text(), 'Book') or contains(text(), 'Réserver') or contains(text(), 'Select') or contains(text(), 'Choose')]")
        
        if moi_buttons:
            moi_button = moi_buttons[0]
            
            print(f"✅ Bouton trouvé près de Moi: '{moi_button.text.strip()}'")
            print(f"🖱️ Clic pour réserver pour Moi...")

            moi_button.click()
                        
            print("✅ Réservation Moi cliquée!")
            time.sleep(time_sleep)
                        
                        # Screenshot final
            driver.save_screenshot(f"7_before_basket_{target_date.replace('-', '_')}.png")
            print(f"📸 Screenshot final: reservation_moi_{target_date.replace('-', '_')}.png")
            return True
    return False
def look_for_slots(driver, target_date, target_level, target_time):
    print("🔍 Recherche de boutons avec données de cours...")

    # Chercher tous les boutons avec data-class-time (structure de votre site)
    course_buttons = driver.find_elements(By.XPATH, "//button[@data-class-time]")

    print(f"📋 {len(course_buttons)} bouton(s) de cours trouvé(s)")

    found_slots = None

    for i, button in enumerate(course_buttons):
        
            # Extraire les données du bouton
            class_name = button.get_attribute('data-class-name') or ''
            class_date = button.get_attribute('data-class-date') or ''
            class_time = button.get_attribute('data-class-time') or ''
            class_spaces = button.get_attribute('data-class-spaces') or '0'
            class_location = button.get_attribute('data-class-location') or ''
            class_venue = button.get_attribute('data-class-venue') or ''
            
            #print(f"\n  Cours {i+1}:")
            #print(f"    Nom: {class_name}")
            #print(f"    Date: {class_date}")
            #print(f"    Heure: {class_time}")
            #print(f"    Places: {class_spaces}")
            
            # Vérifier si c'est le bon jour ET contient 6:30 PM
            date_match = target_date in class_date
            level_match = target_level in class_name
            time_match = target_time in class_time.split("-")[0] or target_time.lstrip() in class_time.split("-")[0]
            
            if date_match:
                 pass
                #print(f"    Heure: {class_time}")
                #print(f"    Nom: {class_name}")
                #print(f"    Date: {class_date}")
                #print(f"    Heure: {class_time}")
                #print(f"    Places: {class_spaces}")
            
            if date_match and time_match and level_match:
                print(f"    ✅ CORRESPONDANCE TROUVÉE!")
                spaces_available = int(class_spaces)
                print(spaces_available)
                #driver.save_screenshot("7_find_date_planning.png")
                slot_info = {
                            'button': button,
                            'class_name': class_name,
                            'class_date': class_date,
                            'class_time': class_time,
                            'class_spaces': spaces_available,
                            'class_location': class_location,
                            'class_venue': class_venue,
                            'available': spaces_available > 0
                        }
                        
                found_slots = slot_info
                        
                if spaces_available > 0:
                    print(f"    ✅ DISPONIBLE - {spaces_available} place(s)")
                    
                else:
                    print(f"    ❌ COMPLET - 0 place")
                break
    if found_slots == None:
        print(f"    ❌ NO CORRESPONDANCE TROUVÉE!")
    return found_slots
            

def click_confirm_basket(driver):
    basket = driver.find_elements(By.XPATH, "//*[contains(text(), 'Checkout')]")
    if basket:
        basket[0].click()
        time.sleep(time_sleep)
        driver.save_screenshot(f"8_after_basket_{TARGET_DATE.replace('-', '_')}.png")
        return True
    return False
                
def click_on_slot(driver, slot_available):
    info_button = slot_available['button']
    parent = info_button.find_element(By.XPATH, "./..")
    book_buttons = parent.find_elements(By.XPATH, ".//*[contains(text(), 'Book Now') or contains(text(), 'Book')]")
    if book_buttons:
        print(f"✅ Bouton 'Book Now' trouvé dans le même conteneur")
        book_buttons[0].click()
        time.sleep(time_sleep)
        #driver.save_screenshot("6_click_on_slot.png")
        return True
    return False


driver = access_login(logon_url, email, password) # navigate after login
possible_to_book, next_week_booking = check_date(TARGET_DATE)

success = False
if possible_to_book:
    if check_basket(driver, login_url): #check empty basket
        driver.get(planning_url) #go to planning after login
       
        if next_week_booking:
            next_week_btn = driver.find_element(By.XPATH, "//input[@value='Next Week']")
            next_week_btn.click()
        time.sleep(time_sleep)  # Attendre le chargement
        # Screenshot
        #driver.save_screenshot("5_planning_page.png")
        print("📸 Screenshot: planning_page.png")
        slot_available = look_for_slots(driver, TARGET_DATE, COURSE_LEVEL, TARGET_TIME)
        if slot_available:
            if click_on_slot(driver, slot_available):
                if click_for_me(driver, TARGET_DATE, my_name):
                    click_confirm_basket(driver)
                    success = True

driver.quit()

if success:
    sys.exit(0)  # Success
else:
    sys.exit(1)